In [1]:
!hostnamectl
!nvidia-smi

 Static hostname: roberto-Lenovo-YOGA-720-13IKB
       Icon name: computer-convertible
         Chassis: convertible
      Machine ID: d4eca54447264437976c8a5dd2499661
         Boot ID: 11ea4f15aec840718abec8af549b2f66
Operating System: ]8;;https://www.ubuntu.com/Ubuntu 24.04.3 LTS]8;;              
          Kernel: Linux 6.14.0-28-generic
    Architecture: x86-64
 Hardware Vendor: Lenovo
  Hardware Model: Lenovo YOGA 720-13IKB
Firmware Version: 1YCN37WW(V2.04)
   Firmware Date: Wed 2017-11-15
    Firmware Age: 7y 10month 2w 6d                
/bin/bash: línea 1: nvidia-smi: orden no encontrada


### La librería Transformers

*Este tutorial es una adaptación de:* https://huggingface.co/docs/transformers/v4.47.1/es/quicktour

La forma más sencilla de experimentar con la librería **Transformers** es a través de la función *pipeline*. Ésta permite ejecutar la tarea de inferencia de un modelo


In [1]:
from transformers import pipeline
import gradio as gr

# Crear el clasificador de análisis de sentimientos
classifier = pipeline('sentiment-analysis')
print(classifier('I love you')[0]['label'])
 

/home/roberto/.virtualenvs/transformers/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu


POSITIVE


También podemos procesar más de una entrada

In [2]:
results = classifier(["We are very happy to show you the 🤗 Transformers library.", "We hope you don't hate it."])
for result in results:
    print(f"label: {result['label']}, with score: {round(result['score'], 4)}")

label: POSITIVE, with score: 0.9998
label: NEGATIVE, with score: 0.5309


También podemos procesar un *dataset* completo 

In [5]:
%pip install soundfile librosa
import torch
from transformers import pipeline
speech_recognizer = pipeline("automatic-speech-recognition", model="facebook/wav2vec2-base-960h")
from datasets import load_dataset, Audio
dataset = load_dataset("PolyAI/minds14", name="en-US", split="train")
dataset = dataset.cast_column("audio", Audio(sampling_rate=speech_recognizer.feature_extractor.sampling_rate))
result = speech_recognizer(dataset[:4]["audio"])
print([d["text"] for d in result])

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Note: you may need to restart the kernel to use updated packages.


Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Generating train split: 0 examples [00:00, ? examples/s]

Dataset minds14 downloaded and prepared to /mnt/netapp2/Store_uni/home/ulc/es/dac/HFCache/datasets/PolyAI___minds14/en-US/1.0.0/7bdf27fccad4efc6da7c00f739d96bfa0d25a7a85f2244755a73c865f70182bc. Subsequent calls will reuse this data.
['I WOULD LIKE TO SET UP A JOINT ACCOUNT WITH MY PARTNER HOW DO I PROCEED WITH DOING THAT', "FONDERING HOW I'D SET UP A JOIN TO HELL T WITH MY WIFE AND WHERE THE AP MIGHT BE", "I I'D LIKE TOY SET UP A JOINT ACCOUNT WITH MY PARTNER I'M NOT SEEING THE OPTION TO DO IT ON THE APSO I CALLED IN TO GET SOME HELP CAN I JUST DO IT OVER THE PHONE WITH YOU AND GIVE YOU THE INFORMATION OR SHOULD I DO IT IN THE AP AN I'M MISSING SOMETHING UQUETTE HAD PREFERRED TO JUST DO IT OVER THE PHONE OF POSSIBLE THINGS", 'HOW DO I FURN A JOINA COUT']


## Autoclasses

In [6]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import AutoTokenizer

model_name= "nlptown/bert-base-multilingual-uncased-sentiment"
model=AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
encoding=tokenizer("I love you")
print(encoding)
pt_batch = tokenizer(["I love you", "I hate you"], padding=True, truncation=True, return_tensors="pt")
pt_outputs = model(**pt_batch)  
from torch import nn
pt_predictions = nn.functional.softmax(pt_outputs.logits, dim=-1)
print(pt_predictions)

{'input_ids': [101, 1045, 2293, 2017, 102], 'token_type_ids': [0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1]}
tensor([[0.1631, 0.1457, 0.2296, 0.2280, 0.2336],
        [0.1635, 0.1629, 0.2798, 0.2095, 0.1843]], grad_fn=<SoftmaxBackward0>)


## Entrenamiento

In [7]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

from datasets import load_dataset
dataset = load_dataset("rotten_tomatoes")

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

dataset = dataset.map(tokenize_function, batched=True)

from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer)

from transformers import Trainer
trainer=Trainer(
    model=model,
    data_collator=data_collator,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    tokenizer=tokenizer,
)
trainer.train()

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/8530 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1066 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1066 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /mnt/netapp2/Store_uni/home/ulc/es/dac/HFCache/datasets/parquet/rotten_tomatoes-ed76130f42b6916d/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7. Subsequent calls will reuse this data.


...

Map:   0%|          | 0/1066 [00:00<?, ? examples/s]

Map:   0%|          | 0/1066 [00:00<?, ? examples/s]

/mnt/netapp2/Store_uni/home/ulc/es/dac/mytx/lib/python3.7/site-packages/transformers/optimization.py:415: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
500,0.717400
1000,0.701700


KeyboardInterrupt: 

### Ejercicio

Haz un código en el que entrenes el mismo modelo pero para el dataset "yelp_review_full"

In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset
from transformers import DataCollatorWithPadding
import torch

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
dataset = load_dataset("yelp_review_full")

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

dataset = dataset.map(tokenize_function, batched=True)


num_labels = 5
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=num_labels)

data_collator = DataCollatorWithPadding(tokenizer)

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    per_device_train_batch_size=8, 
    per_device_eval_batch_size=8,
    num_train_epochs=1,  
    logging_dir="./logs",
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset["train"].shuffle(seed=42).select(range(1000)),  
    eval_dataset=dataset["test"].shuffle(seed=42).select(range(500)),  
    tokenizer=tokenizer,
)

trainer.train()



Found cached dataset parquet (/mnt/netapp2/Store_uni/home/ulc/es/dac/HFCache/datasets/parquet/yelp_review_full-66f1f8c8d1a2da02/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7)


  0%|          | 0/2 [00:00<?, ?it/s]

Loading cached processed dataset at /mnt/netapp2/Store_uni/home/ulc/es/dac/HFCache/datasets/parquet/yelp_review_full-66f1f8c8d1a2da02/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7/cache-f368f8e715b5cf63.arrow
Loading cached processed dataset at /mnt/netapp2/Store_uni/home/ulc/es/dac/HFCache/datasets/parquet/yelp_review_full-66f1f8c8d1a2da02/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7/cache-1d24dd191ad3ca29.arrow
Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expe

Epoch,Training Loss,Validation Loss
1,No log,1.090671


AttributeError: 'list' object has no attribute 'to'

In [ ]:
def test_model(model, tokenizer, test_texts):
    model.eval() 
    inputs = tokenizer(test_texts, padding=True, truncation=True, return_tensors="pt")
    
    with torch.no_grad():
        outputs = model(**inputs)
        predictions = torch.argmax(outputs.logits, dim=-1)
    
    return predictions.tolist()

test_texts = [
    "The food was not the best I have tried",
    "Tomatoes were green, but like they were rotten. Amazing!"
]

device = torch.device("cpu")
model = model.to(device)
test_texts = test_texts

predictions = test_model(model, tokenizer, test_texts)

for text, pred in zip(test_texts, predictions):
    print(f"Review: {text}\nPredicted Sentiment: {pred}\n")

Review: The food was not the best I have tried
Predicted Sentiment: 0

Review: Tomatoes were green, but like they were rotten. Amazing!
Predicted Sentiment: 3

